In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [2]:
df_festival = pd.read_csv('./data/festivals.csv', keep_default_na=False)
df_festival.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...
3,3,Chicken with Vinegar,Poulet au vinaigre,Claude Chabrol,France,0,cannes,1985,https://en.wikipedia.org/wiki/Chicken_with_Vin...,https://en.wikipedia.org/wiki/Claude_Chabrol
4,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...


In [3]:
df_film = pd.read_csv('./data/films.csv', keep_default_na=False)
df_film.head()

,id,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings
0,0,Adieu Bonaparte,1985,N/A,17 May 1985,115 min,"Drama, History, War",Youssef Chahine,"Youssef Chahine, Yusri Nasrullah","Michel Piccoli, Mohsen Mohieddin, Patrice Chér...",...,N/A,N/A,N/A,True,6.4/10,N/A,N/A,,,
1,1,Birdy,1984,R,21 Dec 1984,120 min,"Drama, War",Alan Parker,"William Wharton (based on the novel by), Sandy...","Matthew Modine, Nicolas Cage, John Harkins, Sa...",...,N/A,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,
2,2,Bliss,1985,R,21 Feb 1986,112 min,"Comedy, Drama",Ray Lawrence,"Ray Lawrence (screenplay), Peter Carey (screen...","Barry Otto, Lynette Curran, Helen Jones, Miles...",...,N/A,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,
3,3,,,,,,,,,,...,,,,False,,,,Movie not found!,,
4,4,The Coca-Cola Kid,1985,R,14 Jul 1985,98 min,"Comedy, Drama",Dusan Makavejev,"Frank Moorhouse (screenplay by), Denny Lawrenc...","Eric Roberts, Greta Scacchi, Bill Kerr, Chris ...",...,N/A,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,


In [4]:
df = df_festival.merge(df_film, on=['id'], how='inner')
df.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director,...,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine,...,N/A,N/A,N/A,True,6.4/10,N/A,N/A,,,
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker,...,N/A,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...,...,N/A,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,
3,3,Chicken with Vinegar,Poulet au vinaigre,Claude Chabrol,France,0,cannes,1985,https://en.wikipedia.org/wiki/Chicken_with_Vin...,https://en.wikipedia.org/wiki/Claude_Chabrol,...,,,,False,,,,Movie not found!,,
4,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...,...,N/A,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,


In [5]:
df = df[~df['Plot'].isin(['', 'N/A'])].reset_index(drop=True)
df.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director,...,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine,...,N/A,N/A,N/A,True,6.4/10,N/A,N/A,,,
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker,...,N/A,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...,...,N/A,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,
3,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...,...,N/A,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,
4,5,Colonel Redl,Oberst Redl,István Szabó,Hungary,0,cannes,1985,https://en.wikipedia.org/wiki/Colonel_Redl,https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...,...,N/A,N/A,N/A,True,7.6/10,N/A,N/A,,,


In [6]:
plots = df['Plot']
plots.head()

0    A story during the French Occupation of Egypt ...
1    Philly boys Al and Birdy became friends in hig...
2    An advertising executive dies and goes to hell...
3    An eccentric marketing guru visits a Coca-Cola...
4    Set during the fading glory of the Austro-Hung...
Name: Plot, dtype: object

### preprocessing

In [7]:
plots = [plot.lower() for plot in plots] # lowercase
plots = [re.sub('[!#$\(\)*+,./:;<=>?@^_`|~]', '', plot) for plot in plots] # remove punctuation
plots = [re.sub('\w*\d\w*', '', plot) for plot in plots] # remove numbers and words that contain numbers
plots = [re.sub('-', ' ', plot) for plot in plots] # handle compound words
plots = [word_tokenize(plot) for plot in plots] # tokenize # reevaluate
stopwords = set(stopwords.words('english'))
stopwords.update({'``', '\'\'', '\'s', 'n\'t', '\''})
plots = [
    [word for word in plot if word not in stopwords] # remove stopwords
    for plot in plots
]
plots
df['bag_of_words'] = plots
df.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director,...,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings,bag_of_words
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine,...,N/A,N/A,True,6.4/10,N/A,N/A,,,,"[story, french, occupation, egypt, depicting, ..."
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker,...,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,,"[philly, boys, al, birdy, became, friends, hig..."
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...,...,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,,"[advertising, executive, dies, goes, hell, exc..."
3,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...,...,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,,"[eccentric, marketing, guru, visits, coca, col..."
4,5,Colonel Redl,Oberst Redl,István Szabó,Hungary,0,cannes,1985,https://en.wikipedia.org/wiki/Colonel_Redl,https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...,...,N/A,N/A,True,7.6/10,N/A,N/A,,,,"[set, fading, glory, austro, hungarian, empire..."


### LDA

In [8]:
dictionary = Dictionary(df['bag_of_words'])
dictionary.filter_extremes(no_below = 10) # try other values for no_below
lda_corpus = df['bag_of_words'].apply(dictionary.doc2bow)
print(dictionary)

Dictionary(2530 unique tokens: ['conflict', 'fighting', 'french', 'occupation', 'story']...)


In [9]:
lda = LdaModel(corpus=lda_corpus, id2word=dictionary, num_topics=15)

In [10]:
lda.print_topics()

[(0,
  '0.009*"life" + 0.008*"man" + 0.007*"wife" + 0.007*"new" + 0.006*"back" + 0.006*"story" + 0.006*"old" + 0.006*"lives" + 0.006*"family" + 0.005*"young"'),
 (1,
  '0.013*"young" + 0.009*"one" + 0.009*"life" + 0.009*"man" + 0.007*"family" + 0.006*"get" + 0.006*"husband" + 0.006*"house" + 0.006*"years" + 0.006*"town"'),
 (2,
  '0.016*"one" + 0.012*"old" + 0.011*"town" + 0.010*"day" + 0.009*"young" + 0.009*"year" + 0.008*"small" + 0.008*"love" + 0.007*"time" + 0.006*"mother"'),
 (3,
  '0.012*"love" + 0.011*"old" + 0.010*"life" + 0.010*"film" + 0.009*"de" + 0.008*"father" + 0.008*"la" + 0.006*"two" + 0.006*"year" + 0.006*"one"'),
 (4,
  '0.013*"life" + 0.010*"love" + 0.008*"young" + 0.008*"man" + 0.007*"woman" + 0.007*"son" + 0.007*"two" + 0.007*"mother" + 0.007*"story" + 0.007*"one"'),
 (5,
  '0.013*"father" + 0.011*"one" + 0.011*"family" + 0.010*"life" + 0.008*"old" + 0.007*"man" + 0.007*"young" + 0.006*"son" + 0.005*"tries" + 0.005*"story"'),
 (6,
  '0.011*"years" + 0.010*"young" +

In [11]:
for i in range(15):
    print('Terms: ' + ', '.join([term for term, _ in lda.show_topic(i)]))
    print()

Terms: life, man, wife, new, back, story, old, lives, family, young

Terms: young, one, life, man, family, get, husband, house, years, town

Terms: one, old, town, day, young, year, small, love, time, mother

Terms: love, old, life, film, de, father, la, two, year, one

Terms: life, love, young, man, woman, son, two, mother, story, one

Terms: father, one, family, life, old, man, young, son, tries, story

Terms: years, young, family, life, wife, woman, son, old, man, two

Terms: one, young, girl, son, two, life, world, man, group, french

Terms: two, mother, film, one, wife, son, young, time, man, life

Terms: life, love, children, story, man, wife, young, new, two, woman

Terms: war, world, life, one, german, young, woman, man, love, story

Terms: love, young, new, man, life, wife, anna, meets, one, woman

Terms: life, love, new, mother, relationship, young, son, find, becomes, man

Terms: young, man, life, find, film, goes, mother, one, men, story

Terms: one, father, world, day, you